<a href="https://colab.research.google.com/github/profteachkids/CHE3023/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [49]:
import jax.numpy as jnp
import jax
import tools.che as che
R=8.314

In [18]:
p=che.Props(['Ethanol','Isopropanol','Water'])
T=300
x=jnp.array([1/3,1/3,1/3])

In [19]:
tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T

In [39]:
G = jnp.exp(-p.NRTL_alpha * tau)
xG=jnp.matmul(x,G)
xtauGdivxG = jnp.matmul(x,tau * G) / xG
lngamma = xtauGdivxG+jnp.matmul(G * (tau - xtauGdivxG), x/xG)
jnp.exp(lngamma)

DeviceArray([1.11669551, 1.12519412, 1.85757473], dtype=float64)

In [62]:
def Gex(x,T):
  tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
  G = jnp.exp(-p.NRTL_alpha * tau)
  return jnp.dot(x, jnp.matmul(x,tau*G)/jnp.matmul(x,G))

In [64]:
jnp.exp(jax.grad(Gex,0)(x,T))

DeviceArray([1.11669551, 1.12519412, 1.85757473], dtype=float64)